In [1]:
!pip install mediapipe

     |████████████████████████████████| 32.8 MB 7.2 MB/s            
     |████████████████████████████████| 66.5 MB 26.3 MB/s            


In [1]:
import cv2
import mediapipe as mp
import numpy as np
mp_drawing = mp.solutions.drawing_utils
# We want the pose estimation
mp_pose = mp.solutions.pose

In [2]:
def three_joint_angle(a, b, c):
    """
    This function takes 3 points and form an angle over those.
    Arctan can convert x,y plane into an angle and a radius: arctan(y/x).
    Necessary to change radians to angles
    Parameters
    ----------
    a : int
        First point 
    b : int
        Second point
    c : int
        Third point
        
    Returns
    -------
    float
        Angle of the positions
    """
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)
    
    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(radians * 180.0 / np.pi)
    
    if angle > 180:
        angle = 360 - angle
    
    return(angle)

In [4]:
### Image
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    frame = cv2.imread("../help/img/bad_posture.png")
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    results = pose.process(image)
    
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    
    mp_drawing.draw_landmarks(image,
                              results.pose_landmarks, 
                              mp_pose.POSE_CONNECTIONS, 
                              mp_drawing.DrawingSpec(color=(245, 117, 66), 
                                                     thickness=2, 
                                                     circle_radius=2))
    
    cv2.imshow('', image)
    cv2.waitKey()
    cv2.destroyAllWindows()

In [3]:
### Webcam
#cap = cv2.VideoCapture(0)

### Video
cap = cv2.VideoCapture('../videos/rowing4.mp4')
position = "right" #"left"
# legs straight: 1, otherwise 0.
legs_track = []
# arms straight: 1, otherwise 0.
arms_track = []
# This values might be needed to tweak
min_legs_angle_threshold = 160
min_arms_angle_threshold = 150


with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:  
    while cap.isOpened():
        ret, frame = cap.read()
        
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
                
        results = pose.process(image)
        
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        try:
            landmarks = results.pose_landmarks.landmark
            if position == "left":
                # Extract left part of the body
                hip = landmarks[23]
                knee = landmarks[25]
                ankle = landmarks[27]
                shoulder = landmarks[11]
                elbow = landmarks[13]
                wrist = landmarks[15]
            else:
                # Extract right part of the body
                hip = landmarks[24]
                knee = landmarks[26]
                ankle = landmarks[28]
                shoulder = landmarks[12]
                elbow = landmarks[14]
                wrist = landmarks[16]
                
            hip_visibility = hip.visibility
            knee_visibility = knee.visibility
            ankle_visibility = ankle.visibility
            shoulder_visibility = shoulder.visibility
            elbow_visibility = elbow.visibility
            wrist_visibility = wrist.visibility
                        
            if (hip_visibility >= 0.4 and 
                knee_visibility >= 0.4 and 
                ankle_visibility >= 0.4 and
                shoulder_visibility >= 0.4 and 
                elbow_visibility >= 0.4 and 
                wrist_visibility >= 0.4):
                # Only calculate if visbility
                hip = [hip.x, hip.y]
                knee = [knee.x, knee.y]
                ankle = [ankle.x, ankle.y]
                shoulder = [shoulder.x, shoulder.y]
                elbow = [elbow.x, elbow.y]
                wrist = [wrist.x, wrist.y]

                # Extract angles
                angle_leg = three_joint_angle(hip, knee, ankle)
                angle_arm = three_joint_angle(shoulder, elbow, wrist)
                
                if angle_leg >= min_legs_angle_threshold:
                    # legs straight
                    legs_track.append(1)
                else:
                    legs_track.append(0)
                
                if angle_arm >= min_arms_angle_threshold:
                    # arms straight
                    arms_track.append(1)
                else:
                    arms_track.append(0)
            
                if len(legs_track) > 2:
                    if legs_track[-1] != legs_track[-2] and legs_track[-1] != legs_track[-3]:
                        # Would keep track of instant transition
                        # A stroke would be completed by 2 transitions, going backward and then forward.
                        print("transition")
                        legs_track = []
                    else:
                        # If legs not straight and arms not straight: ERROR
                        if len(arms_track) > 2:
                            if arms_track[-1] != arms_track[-2] and arms_track[-1] != arms_track[-3]:
                                # Would keep track of instant transition
                                # A stroke would be completed by 2 transitions, going backward and then forward.
                                # print("transition")
                                arms_track = []
                            elif legs_track[-1] == 0 and arms_track[-1] == 0:
                                # If legs not straight and arm not straight then warning
                                print("Warning")
                    # Only when legs straight you should pull elbows back
                    # 0 and 1 legs not straight and arms straight
                    # 1 and 0 legs straight and arms not straight
                    # 1 and 1 legs straight and arms straight (should be instantaneous)
                    # 0 and 0 legs not straight and arms not straight warning
                                            
            cv2.putText(image, str(angle_leg), tuple(np.multiply(hip, [1280, 720]).astype(int)), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
            cv2.putText(image, str(angle_arm), tuple(np.multiply(elbow, [1280, 720]).astype(int)), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)

        except:
            pass
        
        
        mp_drawing.draw_landmarks(image, 
                                  results.pose_landmarks, 
                                  mp_pose.POSE_CONNECTIONS, 
                                  mp_drawing.DrawingSpec(color=(245, 117, 66), 
                                                         thickness=2, 
                                                         circle_radius=2))
        
        cv2.imshow("mediapipe test", image)
        if cv2.waitKey(10) & 0xFF == ord("q"):
            break
    
    cap.release()
    cv2.destroyAllWindows()

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


transition
transition
transition
transition
transition
transition
transition
transition
transition
transition
transition
